In [13]:
import torch
from tqdm.auto import tqdm
from transformer_lens import HookedTransformer
from tqdm.auto import tqdm
import plotly.io as pio
import numpy as np
import random
import torch.nn as nn
import torch.nn.functional as F
import wandb
import plotly.express as px
import pandas as pd
import torch.nn.init as init
import pickle
import os
from pathlib import Path
from jaxtyping import Int, Float
from torch import Tensor
import einops
import json
from collections import Counter
from datasets import load_dataset


pio.renderers.default = "notebook_connected"
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
torch.autograd.set_grad_enabled(False)
torch.set_grad_enabled(False)

import sys
sys.path.append('../')  # Add the parent directory to the system path
import utils.haystack_utils as haystack_utils
from sparse_coding.train_autoencoder import AutoEncoder
from utils.autoencoder_utils import custom_forward, AutoEncoderConfig, evaluate_autoencoder_reconstruction

%reload_ext autoreload
%autoreload 2

In [2]:
val_ds = load_dataset("roneneldan/TinyStories", split="validation")
len(val_ds)

Repo card metadata block was not found. Setting CardData to empty.


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

21990

In [12]:
prompts = [x["text"] for x in val_ds]

In [15]:
save_name = "1_ominous_wight"  # "25_gallant_monkey"
model_name = "tiny-stories-1M"

with open(f"{model_name}/{save_name}.json", "r") as f:
    cfg = json.load(f)

cfg = AutoEncoderConfig(
    cfg["layer"], cfg["act"], cfg["expansion_factor"], cfg["l1_coeff"]
)

if cfg.act_name == "hook_mlp_out":
    d_in = model.cfg.d_model  # d_mlp
else:
    d_in = model.cfg.d_mlp
d_hidden = d_in * cfg.expansion_factor

encoder = AutoEncoder(d_hidden, cfg.l1_coeff, d_in)
encoder.load_state_dict(torch.load(os.path.join(path, save_name + ".pt")))
encoder.to(device)

cfg

AutoEncoderConfig(layer=4, act_name='mlp.hook_post', expansion_factor=8, l1_coeff=0.0001)

In [10]:
model = HookedTransformer.from_pretrained(
        model_name,
        center_unembed=True,
        center_writing_weights=True,
        fold_ln=True,
        device=device,
    )


Loaded pretrained model tiny-stories-1M into HookedTransformer


In [16]:
evaluate_autoencoder_reconstruction(encoder, cfg.encoder_hook_point, prompts[:200], model)

100%|██████████| 200/200 [00:06<00:00, 30.00it/s]


(1.8857717996835708, 2.5803273731470107)

In [ ]:
activations = []
for prompt in prompts[:2]:
    _, cache = model.run_with_cache(prompt, names_filter=cfg.encoder_hook_point)
    acts = cache[cfg.encoder_hook_point].squeeze(0)
    _, _, mid_acts, _, _ = encoder(acts)
    print(mid_acts.shape)
    max_prompt_activation = mid_acts.max(0)
    print(max_prompt_activation.shape) # d_enc
    activations.append(max_prompt_activation)

max_activation_per_prompt = torch.stack(activations)
print(max_activation_per_prompt.shape) # n_prompt x d_enc